In [1]:
import sys
sys.path.append('../xor_neuron')

import pickle
from glob import glob
import os
import yaml
from easydict import EasyDict as edict
import numpy as np
import torch
from scipy.signal import convolve2d, fftconvolve, convolve
from scipy.stats import multivariate_normal

import matplotlib.pyplot as plt

from model import *
from utils.train_helper import load_model

In [59]:
dirs = glob('../exp/xor_neuron_conv_cifar/*')
dirs.sort(key=os.path.abspath)
dirs = dirs[:-1]
dirs

['../exp/xor_neuron_conv_cifar/ComplexNeuronConv_001_cifar10_2021-Apr-20-07-17-22',
 '../exp/xor_neuron_conv_cifar/ComplexNeuronConv_001_cifar10_2021-Apr-20-07-17-25',
 '../exp/xor_neuron_conv_cifar/ComplexNeuronConv_001_cifar10_2021-Apr-21-02-58-09']

In [78]:
dirs = glob('../exp/CIFAR100/xor_neuron_mlp/*')
dirs.sort(key=os.path.abspath)
dirs

['../exp/CIFAR100/xor_neuron_mlp/ComplexNeuronMLP_1_cifar100_0727',
 '../exp/CIFAR100/xor_neuron_mlp/ComplexNeuronMLP_2_cifar100_0727']

In [82]:
dirs = []

In [84]:
dirs2 =  glob('../exp/3D_ARG/MLP_MNIST/*')
dirs2.sort(key=os.path.abspath)
# dirs2 = dirs2[1:-1]
dirs2

['../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_10_mnist_0800',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_11_mnist_1752',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_12_mnist_0336',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_13_mnist_0800',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_14_mnist_1744',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_15_mnist_0315',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_16_mnist_0800',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_17_mnist_1741',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_18_mnist_0315',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_19_mnist_0800',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_20_mnist_1744',
 '../exp/3D_ARG/MLP_MNIST/ComplexNeuronMLP_21_mnist_0323']

In [85]:
dirs3 =  glob('../exp/MLP_3D_arg/MNIST/*')
dirs3.sort(key=os.path.abspath)
dirs3

['../exp/MLP_3D_arg/MNIST/ComplexNeuronMLP_1_mnist_0858',
 '../exp/MLP_3D_arg/MNIST/ComplexNeuronMLP_2_mnist_0858',
 '../exp/MLP_3D_arg/MNIST/ComplexNeuronMLP_3_mnist_0635',
 '../exp/MLP_3D_arg/MNIST/ComplexNeuronMLP_4_mnist_0635']

In [80]:
dirs3 =  glob('../exp/2D_arg/CIFAR100/*')
dirs3.sort(key=os.path.abspath)
dirs3

[]

In [86]:
dirs = dirs + dirs2 + dirs3
len(dirs)

16

In [88]:
for ii, dr in enumerate(dirs):
    config_file = glob(dr + '/*.yaml')[0]
    config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))
    
    if config.model.arg_in_dim == 2:
        dim = '2D'
    elif config.model.arg_in_dim == 3:
        dim = '3D'
        
    if config.model.inner_net == 'mlp':
        inner_net = 'MLP'
    elif config.model.inner_net =='conv':
        inner_net = "Conv"
    

    num = 5
    nb = 101
    
    if dim == '2D':
        x = np.linspace(-num, num, nb)
        y = np.linspace(-num, num, nb)

        xv, yv = np.meshgrid(x, y)

        xy = np.vstack([xv.reshape(-1), yv.reshape(-1)]).T
        
    elif dim == '3D':
        num = 5
        nb = 101
        npr = np.random.RandomState(seed=123)

        x = np.linspace(-num, num, nb)
        y = np.linspace(-num, num, nb)
        z = np.linspace(-num, num, nb)

        xv, yv, zv = np.meshgrid(x, y, z)

        xy = np.vstack([xv.reshape(-1), yv.reshape(-1), zv.reshape(-1)]).T
        
    model_pretrain = InnerNet(config)
    model_phase1 = InnerNet(config)


    load_model(model_pretrain.inner_net, config.model_save + config.pretrain.best_model[0])
    model_pretrain.eval()

    model_snapshot = torch.load(config.model_save + config.train.best_model, map_location=torch.device('cpu'))

    for key in list(model_snapshot['model'].keys()):
        model_snapshot['model'][key.replace('0', 'inner_net')] = model_snapshot['model'].pop(key)

    model_phase1.load_state_dict(model_snapshot["model"], strict=True)
    model_phase1.eval()

    if inner_net == 'MLP':
        out_pretrain = model_pretrain.inner_net(torch.Tensor(xy))
        out_pretrain = out_pretrain.data.numpy()
        
        out_phase1 = model_phase1.inner_net(torch.Tensor(xy))
        
        out_inner = out_phase1.data.numpy()
        result = np.concatenate((xy, out_inner), axis=1)
        
        pickle.dump(result, open(f'./Exp_Data/{dim}/{inner_net}/{config.dataset.name.upper()}/InnerNet_Output/out_{config.seed}.p', 'wb'))
        
    elif inner_net == "Conv":
        sqrt_batch_size = np.int(np.sqrt(xy.shape[0]))
        assert sqrt_batch_size ** 2 == xy.shape[0]
        out = xy.T.reshape(1, config.model.arg_in_dim, sqrt_batch_size, sqrt_batch_size)

        out_pretrain = model_pretrain.inner_net(torch.Tensor(out))
        out_pretrain = out_pretrain.data.numpy().reshape(-1,1)

        out_phase1 = model_phase1.inner_net(torch.Tensor(out))
    
        out_inner = out_phase1.data.numpy()
        res = out_inner[0]
        mesh = out[0]
        
        result = np.concatenate((mesh, res))
        pickle.dump(result, open(f'./Exp_Data/{dim}/{inner_net}/{config.dataset.name.upper()}/InnerNet_Output/out_{config.seed}.p', 'wb'))
        
    input2innerAll = glob(dr+'/in2cells.p')
    if len(input2innerAll) == 0:
        input2innerAll = glob(dr+'/model_save/in2cells.p')
    input2innerAll = input2innerAll[0]
    input2innerAll = pickle.load(open(input2innerAll, 'rb'))
    input2innerAll = [input2innerAll[k] for k in range(len(input2innerAll))]
    
    if inner_net == 'MLP':
        input2innerAll = np.array(input2innerAll)
    elif inner_net == 'Conv':
        input2innerAll = np.array(input2innerAll[0])
        
    input2innerAll = np.moveaxis(input2innerAll, -1, 0)
    input2innerAll = input2innerAll.reshape((config.model.arg_in_dim, -1))
    
    xedges = yedges = zedges = np.arange(-5.05,5.1,0.1)
    pdf, _, _ = np.histogram2d(input2innerAll[0], input2innerAll[1], bins=(xedges, yedges))
    pdf = convolve2d(pdf, gaussian_kernel, mode='same')
    pdf /= sum(pdf.flatten())
    
    pickle.dump(pdf, open(f'./Exp_Data/{dim}/{inner_net}/{config.dataset.name.upper()}/Input_Distribution/input_distribution_{config.seed}.p', 'wb'))

In [89]:
input2innerAll.shape

(3, 3600000)

In [90]:
pdf.shape

(101, 101)

In [91]:
xy.shape

(1030301, 3)

In [92]:
result.shape

(1030301, 4)

In [93]:
result

array([[-5.        , -5.        , -5.        ,  1.11572051],
       [-5.        , -5.        , -4.9       ,  1.15741193],
       [-5.        , -5.        , -4.8       ,  1.199103  ],
       ...,
       [ 5.        ,  5.        ,  4.8       ,  0.34355482],
       [ 5.        ,  5.        ,  4.9       ,  0.38146681],
       [ 5.        ,  5.        ,  5.        ,  0.419379  ]])

In [95]:
xedges = yedges = zedges = np.arange(-5.05,5.1,0.1)

In [102]:
xedges

array([-5.05, -4.95, -4.85, -4.75, -4.65, -4.55, -4.45, -4.35, -4.25,
       -4.15, -4.05, -3.95, -3.85, -3.75, -3.65, -3.55, -3.45, -3.35,
       -3.25, -3.15, -3.05, -2.95, -2.85, -2.75, -2.65, -2.55, -2.45,
       -2.35, -2.25, -2.15, -2.05, -1.95, -1.85, -1.75, -1.65, -1.55,
       -1.45, -1.35, -1.25, -1.15, -1.05, -0.95, -0.85, -0.75, -0.65,
       -0.55, -0.45, -0.35, -0.25, -0.15, -0.05,  0.05,  0.15,  0.25,
        0.35,  0.45,  0.55,  0.65,  0.75,  0.85,  0.95,  1.05,  1.15,
        1.25,  1.35,  1.45,  1.55,  1.65,  1.75,  1.85,  1.95,  2.05,
        2.15,  2.25,  2.35,  2.45,  2.55,  2.65,  2.75,  2.85,  2.95,
        3.05,  3.15,  3.25,  3.35,  3.45,  3.55,  3.65,  3.75,  3.85,
        3.95,  4.05,  4.15,  4.25,  4.35,  4.45,  4.55,  4.65,  4.75,
        4.85,  4.95,  5.05])

In [103]:
H, edges = np.histogramdd([input2innerAll[0], input2innerAll[1], input2innerAll[2]], bins = (xedges, yedges, zedges))

In [104]:
H.shape

(101, 101, 101)